### LLM As Judge
* LLM 모델을 평가자로 설정하여 모델의 성능을 평가하고 개선할 수 있다.

### OFF the shelf Evaluator
* LandSmith에서 제공하는 기본 평가 LLM을 사용해 모델의 출력을 자동으로 평가할 수 있게 해준다.

**주요특징**
* 사전 정의된 평가 기준 제공
* 일관된 평가 방식 적용
* 대규모 출력 평가 자동화 기능

**필요정보**
* input: 질문, 보통 데이터셋의 Question이 사용된다.
* prection : LLM이 생성한 답변
* reference : 정답 답변, Context 등 변칙적으로 사용 가능

In [1]:

from rag import PDFRAG
from langchain_openai import ChatOpenAI

rag = PDFRAG(
    file_path="data/snow-white.pdf", llm=ChatOpenAI(model_name="gpt-4o-mini",temperature=0)
)

retriever = rag.create_retriever()

chain = rag.create_chain(retriever)

chain.invoke("백설공주는 어떤 과일을 먹고 쓰러졌나요?")

'백설공주는 사과를 먹고 쓰러졌습니다.'

In [2]:
# 질문에 답변하는 함수
def ask_question(inputs : dict):
    return {"answer" : chain.invoke(inputs["question"])}

In [4]:
llm_answer = ask_question(
    {"question": "백설공주는 어떤 과일을 먹고 쓰러졌나요?"}
)

llm_answer

{'answer': '백설공주는 사과를 먹고 쓰러졌습니다.'}

In [5]:
# evaluator prompt 출력을 위한 함수
def print_evaluator_prompt(evaluator):
    return evaluator.evaluator.prompt.pretty_print()

### Question-Answer-Evaluator
* 질문(Question)과 답변(Answer)를 평가합니다.

* input: 사용자 입력
* prediction: LLM이 생성한 답변
* reference: 정답 답변

**참고**
Evaluator 프롬프트의 변수에는 query(input), result(prediction), answer(reference)로 정의된다.

In [6]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

# qa 평가자 생성
qa_evaluator = LangChainStringEvaluator("qa")

print_evaluator_prompt(qa_evaluator)

You are a teacher grading a quiz.
You are given a question, the student's answer, and the true answer, and are asked to score the student answer as either CORRECT or INCORRECT.

Example Format:
QUESTION: question here
STUDENT ANSWER: student's answer here
TRUE ANSWER: true answer here
GRADE: CORRECT or INCORRECT here

Grade the student answers based ONLY on their factual accuracy. Ignore differences in punctuation and phrasing between the student answer and true answer. It is OK if the student answer contains more information than the true answer, as long as it does not contain any conflicting statements. Begin! 

QUESTION: {query}
STUDENT ANSWER: {result}
TRUE ANSWER: {answer}
GRADE:


In [ ]:
dataset_name = "RAG_EVALUATION_DATASET"

experiement_results = evaluate(
    ask_question, # 평가할 함수 지정
    data=dataset_name, # 데이터셋 지정
    evaluation=[qa_evaluator]
)

### Context에 기반한 답변 Evaluator
**"context_qa"**
* LLM 체인에 정확성을 판단하는 데 context를 사용하도록 지시

**"cot_qa"**
* 최종 판결을 하기 전에 LLM의 추론을 사용하도록 지시

In [ ]:
# Context를 반환하는 RAG 결과 반환 함수
def rag_context_answer(inputs: dict):
    context = retriever.invoke(inputs["question"])
    return {
        "context": "\n".join([doc.page_content for doc in context]),
        "answer": chain.invoke,
        "query":inputs["question"]
    }

In [ ]:

#데이터셋 이름
dataset_name = "RAG_EVALUATION_DATASET"

# 평가실행
evaluate(
    rag_context_answer,
    data=dataset_name,
    evaluators=[cot_qa_evaluator, context_qa_evaluator],
    experiment_prefix="RAG_EVALUATION",
    metadata={
        "variant" : "COT_QA & CONTEXT_QA Evaluatior 를 활용한 평가"
    }
)

### Criteria
* 기준값이 없거나 얻기 힘든 경우 "criteria"를 통해 사용자 지정 기준 집합에 대한 실행을 평가할 수 있다.
* 답변에 대해 높은 수준의 의미론적 측면을 평가하고 자 할 때 유용하다.

LangChainStringEvaluator("criteria", config={ "criteria": `아래 중 하나의 criterion` })

| 기준 | 설명 |
|------|------|
| `conciseness` | 답변이 간결하고 간단한지 평가 |
| `relevance` | 답변이 질문과 관련이 있는지 평가 |
| `correctness` | 답변이 옳은지 평가 |
| `coherence` | 답변이 일관성이 있는지 평가 |
| `harmfulness` | 답변이 해롭거나 유해한지 평가 |
| `maliciousness` | 답변이 악의적이거나 악화시키는지 평가 |
| `helpfulness` | 답변이 도움이 되는지 평가 |
| `controversiality` | 답변이 논란이 되는지 평가 |
| `misogyny` | 답변이 여성을 비하하는지 평가 |
| `criminality` | 답변이 범죄를 촉진하는지 평가 |

In [7]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

# 평가자 설정
criteria_evaluator = [
    LangChainStringEvaluator("criteria", config={"criteria": "conciseness"}),
    LangChainStringEvaluator("criteria", config={"criteria": "relevance"}),
    LangChainStringEvaluator("criteria", config={"criteria": "coherence"})
]

# 데이터셋 이름 설정]
dataset_name = "RAG_EVALUATION_DATASET"

# 평가 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=criteria_evaluator,
    experiment_prefix="CRITERIA_EVALUATION",
    metadata={
        "variant": "criteria를 활용한 평가"
    }
)

View the evaluation results for experiment: 'CRITERIA_EVALUATION-765913e1' at:
https://smith.langchain.com/o/7a65adeb-16f3-4a3a-be13-7b7aab4ae17b/datasets/917e83ee-8f0b-45b5-ba49-ed8d045141f1/compare?selectedSessions=eb7af667-01e6-4d8c-817d-fdd25e157ca0




0it [00:00, ?it/s]